<a href="https://colab.research.google.com/github/chrisguti27/Notebooks/blob/main/AI/Machine%20Learning/Preprocesado%20de%20Datos/DataPrepros_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!cd ..
!ls
!git clone https://github.com/chrisguti27/Notebooks.git

# Importar Datasets

Data set: Conjunto de datos que van a ser usados para desarrollar el algoritmo de machine learning.

In [3]:
import numpy as np
import pandas as  pd #libreria para importar datasets

Data frame: Cuando el *Data set* se presenta en forma una tabla ".csv", o un *array* bidimensional.

In [37]:
dataset = pd.read_csv("Notebooks/AI/Machine Learning/Preprocesado de Datos/Data.csv")
print("--------------Dataset-------------- \n",dataset,
      "\n-----------------------------------")

--------------Dataset-------------- 
    Country   Age   Salary Purchased
0   France  44.0  72000.0        No
1    Spain  27.0  48000.0       Yes
2  Germany  30.0  54000.0        No
3    Spain  38.0  61000.0        No
4  Germany  40.0      NaN       Yes
5   France  35.0  58000.0       Yes
6    Spain   NaN  52000.0        No
7   France  48.0  79000.0       Yes
8  Germany  50.0  83000.0        No
9   France  37.0  67000.0       Yes 
-----------------------------------


El data set, en este caso data frame, se divide en variables independientes o variables predictoras *X* y variables dependientes *Y*.

In [40]:
X = dataset.iloc[:,:-1].values  #.iloc index localization
Y = dataset.iloc[:,-1].values   #-1 ultima columna no se incluye en el dato
                                #.values extraer solo los valores y no las posiciones
print("Variables independientes\n", X,"\n Variables dependientes\n",Y)

Variables independientes
 [['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]] 
 Variables dependientes
 ['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


# Tratamiento de los NAs o datos faltantes/desconocidos

En ciertos casos pueden faltar uno o varios datos en el dataset, es decir que encontraremos valores *NaN*. Considerando que al borrar las filas con valores *NaN* perderíamos información importante para modelar el algoritmo. Es necesario aplicar métodos para asignar un valor correspondiente que reemplace este valor.

In [42]:
from sklearn.impute import SimpleImputer #llama a la funcion Imputer
                                         # biblioteca de preprosemiento de datos
imputer = SimpleImputer(missing_values = np.nan, strategy = "mean", verbose = 0)
#missing_values : valores de las celdas sin informacion
#strategy : estrategia para reemplazar el valor de dicha celda, 
            #"mean" ----> mediana
#verbose: respecto a que eje se va a realizar strategy,
            #0: la accion strategy de la columna
            #1: la accion strategy de la fila
imputer.fit(X[:,1:3])#la ultima columna en el corchete no esta incluido
#se reemplazan los valores NAN por los valores de la funcion imputer
X[:,1:3] = imputer.transform(X[:,1:3])
print("----Data set sin NaNs-----\n",X)

----Data set sin NaNs-----
 [['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


# Datos categóricos

Los datos o variables categóricas son etiquetas, valores cualitativos, es decir valores no ordinales. También se las conoce como variables dummy.

En el procesamiento de datos para machine learning hay que señalar la importancia de manejar datos numéricos para aplicar o desarrollar algoritmos de modelamiento. Por lo que, estas variables dummy deben ser sometidas a un procesamiento para expresar categoría en una representación numérica. Para ello se emplea el determinado *one hot encoding*, que se trata de tranformar la variables, valores de la columna *Country* y *Purchased*, en tantas columnas como categorías existan.


In [43]:
#codificar datos categoricos
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder_X = LabelEncoder()
X[:,0] = labelencoder_X.fit_transform(X[:,0])

#one hot encoding para variales categoricas, sin valor ordinal
ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [0])],    # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough'                         # Leave the rest of the columns untouched
)

X = np.array(ct.fit_transform(X), dtype=np.int32)

print("Codificacion de Datos Categoricos para X\n",X)

labelencoder_Y = LabelEncoder()
Y = labelencoder_Y.fit_transform(Y)
print("Codificacion de Datos Categoricos para Y\n",Y)

Codificacion de Datos Categoricos para X
 [[    1     0     0    44 72000]
 [    0     0     1    27 48000]
 [    0     1     0    30 54000]
 [    0     0     1    38 61000]
 [    0     1     0    40 63777]
 [    1     0     0    35 58000]
 [    0     0     1    38 52000]
 [    1     0     0    48 79000]
 [    0     1     0    50 83000]
 [    1     0     0    37 67000]]
Codificacion de Datos Categoricos para Y
 [0 1 0 0 1 1 0 1 0 1]


# Dividir dataset en entrenamiento y test

Dentro del algoritmo de machine learning se pueden encontrar dos fases, *entretanimiento* y *test*, donde del data set se dispone una determinada cantidad para cada fase, procurando que exista la suficiente cantidad de datos para evitar *overfitting*, es decir, asegurar que exista aprendizaje. El porcentaje óptimo se entra en una realación 20-30%:80-70%, siendo el mayor procentaje para la fase de entrenamiento.

*overfitting*: cuando el modelo se ajusta solamente a los datos de entrenamiento

In [53]:
#Dividir el data set en conjunto de entrenamiento y conjunto de testing

from sklearn.model_selection import train_test_split
#X_train: v. independientes para entrenar el algoritmo
#X_test: v. independientes para testing
#Y_train: v. dependientes para entrenar el algoritmo
#Y_test: v. dependientes para testi
#train_test_split(X, Y, test_size = 0.2)
    #X: matriz de v. indep, 
    #Y: vector de v. a predecir, 
    #test_size = % que se usaran para testing 20-30% para testing es lo normal
    #mientras mas datos para entrenar tenga el algoritmo mejor sera la predicción
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)
print("-------------+++X+++-------------\n","------------Training------------\n",X_train,
      "\n----------------Test-------------\n", X_test, 
      "\n--------------+++Y+++------------\n","------------Training------------\n",Y_train,
      "\n----------------Test-------------\n",Y_test)

-------------+++X+++-------------
 ------------Training------------
 [[    0     1     0    40 63777]
 [    1     0     0    37 67000]
 [    0     0     1    27 48000]
 [    0     0     1    38 52000]
 [    1     0     0    48 79000]
 [    0     0     1    38 61000]
 [    1     0     0    44 72000]
 [    1     0     0    35 58000]] 
----------------Test-------------
 [[    0     1     0    30 54000]
 [    0     1     0    50 83000]] 
--------------+++Y+++------------
 ------------Training------------
 [1 1 1 0 1 0 0 1] 
----------------Test-------------
 [0 0]


# Escalamiento de datos

Los valores dentro de un data set por lo general no se encuentran en el mismo rango numérico, es decir, puede ser que los valores de la variable *edad* esté en el rango de *0-100* mientras que los de purchased en el rango *30000 - 100000*. Por lo tanto, los valores de esta última variable tendrán un impacto mayor en el modelo de lo que los valores de la variable edad.

Para evitar este escenario, que unas variables dominen sobre otras, se realiza un escalado que puede ser mediante  normalización o estandarización de los datos.

Con esto se asegura que el propio algoritmo se encargue de disernir qué peso otorgar a cada variable para el modelado del data set, según se aporte al proceso de clasificación o predicción.

---
Para el caso de este data set, los valores de salida *Y* no se escalan porque el algoritmo es de clasificación.


In [55]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()#escalador de datos por STANDARISATION
#se escala el conjunto de entrenamiento
#con sc_X.fit_transform(X_train) el conjunto de entrenamiento se escala automaticamente
X_train = sc_X.fit_transform(X_train)
print("Variables de entrenamiento escaladas:\n",X_train)
#para las variables test se emplea el mismo escalado que
#para las variables train, por eso solamente se llama
#al metodo sc_X.transform(X_test)
X_test = sc_X.transform(X_test)
print("Variables de test escaladas:\n",X_train)

Variables de entrenamiento escaladas:
 [[-1.          2.64575131 -0.77459667  0.27978024  0.12374357]
 [ 1.         -0.37796447 -0.77459667 -0.23673712  0.4617671 ]
 [-1.         -0.37796447  1.29099445 -1.95846165 -1.53092514]
 [-1.         -0.37796447  1.29099445 -0.06456467 -1.11141099]
 [ 1.         -0.37796447 -0.77459667  1.65715986  1.72030956]
 [-1.         -0.37796447  1.29099445 -0.06456467 -0.16750414]
 [ 1.         -0.37796447 -0.77459667  0.96847005  0.98615979]
 [ 1.         -0.37796447 -0.77459667 -0.58108203 -0.48213975]]
Variables de test escaladas:
 [[-1.          2.64575131 -0.77459667  0.27978024  0.12374357]
 [ 1.         -0.37796447 -0.77459667 -0.23673712  0.4617671 ]
 [-1.         -0.37796447  1.29099445 -1.95846165 -1.53092514]
 [-1.         -0.37796447  1.29099445 -0.06456467 -1.11141099]
 [ 1.         -0.37796447 -0.77459667  1.65715986  1.72030956]
 [-1.         -0.37796447  1.29099445 -0.06456467 -0.16750414]
 [ 1.         -0.37796447 -0.77459667  0.9684700

Dentro de la fase de escalado se halla una discusión, acerca de sí escalar o no las variables dummy. No existe una sóla respuesta, ya que para cualquiera de los casos se puede dar razones de peso para inclinarse por una u otra postura. Todo dependerá del cirterio que quiera abordar el programador, que a su vez estará sujeto a cada situación y problemática que interpreten los datos.

Lo que está claro, es que en cualquiera de las dos situaciones, *Sí* o *No*, la velocidad de convergencia del algoritmo no se verá afectada.